In [1]:
# Problem 1
import requests

url = 'https://pegkq2svv6.execute-api.ap-northeast-2.amazonaws.com/prod/users'

def start(token, problem=1):    
    tmp_url = url + '/start'
    return requests.post(tmp_url, headers = token, json = {"problem": 1}).json()

def locations(auth_key):
    tmp_url = url + '/locations'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def trucks(auth_key):
    tmp_url = url + '/trucks'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def simulate(auth_key, commands = None):
#     commands = [{ "truck_id": 0, "command": [2, 5, 4, 1, 6]}]
    '''
    commands :
        0: 6초간 아무것도 하지 않음
        1: 위로 한 칸 이동
        2: 오른쪽으로 한 칸 이동
        3: 아래로 한 칸 이동
        4: 왼쪽으로 한 칸 이동
        5: 자전거 상차
        6: 자전거 하차
    '''
    tmp_url = url + '/simulate'
    return requests.put(tmp_url, headers = {'Authorization' : auth_key}, json = {"commands" : commands})

def score(auth_key):
    tmp_url = url + '/score'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def change_loc(N,i,j):
    return j, N-i

def next_loc(N, idx):
    i,j = idx[0],idx[1]
    # 지그재그식
    if i == 0 and j == N-1:
        return -1
    if j % 2 == 1:
        if i == N-1:
            return 2
        else:
            return 3
    if j % 2 == 0:
        if i == 0:
            return 2
        else:
            return 1
        
def next_loc_idx(num,idx):
    i,j = idx[0],idx[1]
    if num == 1: return [i-1,j]
    elif num == 2: return [i, j+1]
    elif num == 3: return [i+1, j]
    elif num == 4: return [i, j-1]

def main():
    token = {'X-Auth-Token' : '715f565304b4aaab9d6de793bfc51818'}
    N = 5
    truck = {_ : [4,0] for _ in range(N)}
    
    start_res = start(token, 1)
    print(start_res)
    
    locations_res = locations(start_res['auth_key'])
    # 각 자전거 대여소의 ID, 보유하고 있는 자전거 수에 대한 정보를 담은 배열
    print(locations_res.json())
    
    trucks_res = trucks(start_res['auth_key'])
    # 각 트럭의 ID, 현재 위치, 싣고 있는 자전거 수에 대한 정보를 담은 배열
    print(trucks_res.json())
    
    # 처음 1분은 열별 1대씩 배치
    simulate_res = simulate(start_res['auth_key'], commands = [{"truck_id": 1, "command": [0,1,1,1,1],
                                                               "truck_id": 2, "command": [0, 2, 2],
                                                               "truck_id": 3, "command": [0, 1,1,1,1,2,2],
                                                               "truck_id": 4, "command": [0, 2, 2, 2, 2]}])
    truck[1] = [0,0]
    truck[2] = [4,2]
    truck[3] = [0,2]
    truck[4] = [4,4]
    
    for _ in range(720):
        _dict = [{'truck_id' : _, "command" : []} for _ in range(N)]
        for _ in _dict:
            _['command'] = []
            next_res = next_loc(N, truck[_['truck_id']])
            if next_res == -1:
                _['command'] += [3,4] * 4
                truck[_['truck_id']] = [4,0]
            else:
                _['command'] += [next_res]
                truck[_['truck_id']] = next_loc_idx(next_res, truck[_['truck_id']])
                _['command'] += [5] * 2
                _['command'] += [6] * 1
                
                next_res = next_loc(N, truck[_['truck_id']])
                _['command'] += [next_res]
                truck[_['truck_id']] = next_loc_idx(next_res, truck[_['truck_id']])
                _['command'] += [5] * 2
                _['command'] += [6] * 1
                
        simulate_res = simulate(start_res['auth_key'],_dict)
        print(simulate_res.json())
    
    score_res = score(start_res['auth_key'])
    print(score_res.json())

In [2]:
if __name__ == '__main__':
    main()

{'auth_key': '2ad37efc-a698-459c-93c5-00187a81c6aa', 'problem': 1, 'time': 0}
{'locations': [{'id': 0, 'located_bikes_count': 4}, {'id': 1, 'located_bikes_count': 4}, {'id': 2, 'located_bikes_count': 4}, {'id': 3, 'located_bikes_count': 4}, {'id': 4, 'located_bikes_count': 4}, {'id': 5, 'located_bikes_count': 4}, {'id': 6, 'located_bikes_count': 4}, {'id': 7, 'located_bikes_count': 4}, {'id': 8, 'located_bikes_count': 4}, {'id': 9, 'located_bikes_count': 4}, {'id': 10, 'located_bikes_count': 4}, {'id': 11, 'located_bikes_count': 4}, {'id': 12, 'located_bikes_count': 4}, {'id': 13, 'located_bikes_count': 4}, {'id': 14, 'located_bikes_count': 4}, {'id': 15, 'located_bikes_count': 4}, {'id': 16, 'located_bikes_count': 4}, {'id': 17, 'located_bikes_count': 4}, {'id': 18, 'located_bikes_count': 4}, {'id': 19, 'located_bikes_count': 4}, {'id': 20, 'located_bikes_count': 4}, {'id': 21, 'located_bikes_count': 4}, {'id': 22, 'located_bikes_count': 4}, {'id': 23, 'located_bikes_count': 4}, {'id'

{'status': 'ready', 'time': 88, 'failed_requests_count': 16, 'distance': '106.0'}
{'status': 'ready', 'time': 89, 'failed_requests_count': 16, 'distance': '107.0'}
{'status': 'ready', 'time': 90, 'failed_requests_count': 16, 'distance': '108.6'}
{'status': 'ready', 'time': 91, 'failed_requests_count': 16, 'distance': '109.6'}
{'status': 'ready', 'time': 92, 'failed_requests_count': 16, 'distance': '111.2'}
{'status': 'ready', 'time': 93, 'failed_requests_count': 16, 'distance': '112.2'}
{'status': 'ready', 'time': 94, 'failed_requests_count': 16, 'distance': '113.2'}
{'status': 'ready', 'time': 95, 'failed_requests_count': 16, 'distance': '114.8'}
{'status': 'ready', 'time': 96, 'failed_requests_count': 16, 'distance': '115.8'}
{'status': 'ready', 'time': 97, 'failed_requests_count': 16, 'distance': '116.8'}
{'status': 'ready', 'time': 98, 'failed_requests_count': 16, 'distance': '118.4'}
{'status': 'ready', 'time': 99, 'failed_requests_count': 16, 'distance': '119.4'}
{'status': 'read

{'status': 'ready', 'time': 187, 'failed_requests_count': 39, 'distance': '227.8'}
{'status': 'ready', 'time': 188, 'failed_requests_count': 39, 'distance': '228.8'}
{'status': 'ready', 'time': 189, 'failed_requests_count': 39, 'distance': '230.4'}
{'status': 'ready', 'time': 190, 'failed_requests_count': 40, 'distance': '231.4'}
{'status': 'ready', 'time': 191, 'failed_requests_count': 41, 'distance': '233.0'}
{'status': 'ready', 'time': 192, 'failed_requests_count': 41, 'distance': '234.0'}
{'status': 'ready', 'time': 193, 'failed_requests_count': 41, 'distance': '235.0'}
{'status': 'ready', 'time': 194, 'failed_requests_count': 41, 'distance': '236.6'}
{'status': 'ready', 'time': 195, 'failed_requests_count': 41, 'distance': '237.6'}
{'status': 'ready', 'time': 196, 'failed_requests_count': 41, 'distance': '239.2'}
{'status': 'ready', 'time': 197, 'failed_requests_count': 41, 'distance': '240.2'}
{'status': 'ready', 'time': 198, 'failed_requests_count': 41, 'distance': '241.2'}
{'st

{'status': 'ready', 'time': 286, 'failed_requests_count': 57, 'distance': '349.6'}
{'status': 'ready', 'time': 287, 'failed_requests_count': 59, 'distance': '351.2'}
{'status': 'ready', 'time': 288, 'failed_requests_count': 59, 'distance': '352.2'}
{'status': 'ready', 'time': 289, 'failed_requests_count': 59, 'distance': '353.2'}
{'status': 'ready', 'time': 290, 'failed_requests_count': 59, 'distance': '354.8'}
{'status': 'ready', 'time': 291, 'failed_requests_count': 59, 'distance': '355.8'}
{'status': 'ready', 'time': 292, 'failed_requests_count': 59, 'distance': '356.8'}
{'status': 'ready', 'time': 293, 'failed_requests_count': 59, 'distance': '358.4'}
{'status': 'ready', 'time': 294, 'failed_requests_count': 59, 'distance': '359.4'}
{'status': 'ready', 'time': 295, 'failed_requests_count': 59, 'distance': '361.0'}
{'status': 'ready', 'time': 296, 'failed_requests_count': 59, 'distance': '362.0'}
{'status': 'ready', 'time': 297, 'failed_requests_count': 60, 'distance': '363.0'}
{'st

{'status': 'ready', 'time': 385, 'failed_requests_count': 79, 'distance': '471.4'}
{'status': 'ready', 'time': 386, 'failed_requests_count': 80, 'distance': '473.0'}
{'status': 'ready', 'time': 387, 'failed_requests_count': 81, 'distance': '474.0'}
{'status': 'ready', 'time': 388, 'failed_requests_count': 81, 'distance': '475.0'}
{'status': 'ready', 'time': 389, 'failed_requests_count': 81, 'distance': '476.6'}
{'status': 'ready', 'time': 390, 'failed_requests_count': 81, 'distance': '477.6'}
{'status': 'ready', 'time': 391, 'failed_requests_count': 81, 'distance': '479.2'}
{'status': 'ready', 'time': 392, 'failed_requests_count': 81, 'distance': '480.2'}
{'status': 'ready', 'time': 393, 'failed_requests_count': 81, 'distance': '481.2'}
{'status': 'ready', 'time': 394, 'failed_requests_count': 81, 'distance': '482.8'}
{'status': 'ready', 'time': 395, 'failed_requests_count': 81, 'distance': '483.8'}
{'status': 'ready', 'time': 396, 'failed_requests_count': 81, 'distance': '484.8'}
{'st

{'status': 'ready', 'time': 485, 'failed_requests_count': 103, 'distance': '594.8'}
{'status': 'ready', 'time': 486, 'failed_requests_count': 103, 'distance': '595.8'}
{'status': 'ready', 'time': 487, 'failed_requests_count': 104, 'distance': '596.8'}
{'status': 'ready', 'time': 488, 'failed_requests_count': 104, 'distance': '598.4'}
{'status': 'ready', 'time': 489, 'failed_requests_count': 104, 'distance': '599.4'}
{'status': 'ready', 'time': 490, 'failed_requests_count': 104, 'distance': '601.0'}
{'status': 'ready', 'time': 491, 'failed_requests_count': 105, 'distance': '602.0'}
{'status': 'ready', 'time': 492, 'failed_requests_count': 106, 'distance': '603.0'}
{'status': 'ready', 'time': 493, 'failed_requests_count': 107, 'distance': '604.6'}
{'status': 'ready', 'time': 494, 'failed_requests_count': 107, 'distance': '605.6'}
{'status': 'ready', 'time': 495, 'failed_requests_count': 108, 'distance': '607.2'}
{'status': 'ready', 'time': 496, 'failed_requests_count': 109, 'distance': '

{'status': 'ready', 'time': 583, 'failed_requests_count': 132, 'distance': '715.0'}
{'status': 'ready', 'time': 584, 'failed_requests_count': 132, 'distance': '716.6'}
{'status': 'ready', 'time': 585, 'failed_requests_count': 134, 'distance': '717.6'}
{'status': 'ready', 'time': 586, 'failed_requests_count': 134, 'distance': '719.2'}
{'status': 'ready', 'time': 587, 'failed_requests_count': 134, 'distance': '720.2'}
{'status': 'ready', 'time': 588, 'failed_requests_count': 134, 'distance': '721.2'}
{'status': 'ready', 'time': 589, 'failed_requests_count': 134, 'distance': '722.8'}
{'status': 'ready', 'time': 590, 'failed_requests_count': 134, 'distance': '723.8'}
{'status': 'ready', 'time': 591, 'failed_requests_count': 134, 'distance': '724.8'}
{'status': 'ready', 'time': 592, 'failed_requests_count': 134, 'distance': '726.4'}
{'status': 'ready', 'time': 593, 'failed_requests_count': 134, 'distance': '727.4'}
{'status': 'ready', 'time': 594, 'failed_requests_count': 134, 'distance': '

{'status': 'ready', 'time': 681, 'failed_requests_count': 151, 'distance': '835.8'}
{'status': 'ready', 'time': 682, 'failed_requests_count': 151, 'distance': '836.8'}
{'status': 'ready', 'time': 683, 'failed_requests_count': 151, 'distance': '838.4'}
{'status': 'ready', 'time': 684, 'failed_requests_count': 151, 'distance': '839.4'}
{'status': 'ready', 'time': 685, 'failed_requests_count': 151, 'distance': '841.0'}
{'status': 'ready', 'time': 686, 'failed_requests_count': 152, 'distance': '842.0'}
{'status': 'ready', 'time': 687, 'failed_requests_count': 152, 'distance': '843.0'}
{'status': 'ready', 'time': 688, 'failed_requests_count': 153, 'distance': '844.6'}
{'status': 'ready', 'time': 689, 'failed_requests_count': 153, 'distance': '845.6'}
{'status': 'ready', 'time': 690, 'failed_requests_count': 153, 'distance': '847.2'}
{'status': 'ready', 'time': 691, 'failed_requests_count': 153, 'distance': '848.2'}
{'status': 'ready', 'time': 692, 'failed_requests_count': 153, 'distance': '

In [3]:
# 715f56	783.0	228	37.20

In [4]:
# Problem 1
import requests

url = 'https://pegkq2svv6.execute-api.ap-northeast-2.amazonaws.com/prod/users'

def start(token, problem=1):    
    tmp_url = url + '/start'
    return requests.post(tmp_url, headers = token, json = {"problem": problem}).json()

def locations(auth_key):
    tmp_url = url + '/locations'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def trucks(auth_key):
    tmp_url = url + '/trucks'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def simulate(auth_key, commands = None):
#     commands = [{ "truck_id": 0, "command": [2, 5, 4, 1, 6]}]
    '''
    commands :
        0: 6초간 아무것도 하지 않음
        1: 위로 한 칸 이동
        2: 오른쪽으로 한 칸 이동
        3: 아래로 한 칸 이동
        4: 왼쪽으로 한 칸 이동
        5: 자전거 상차
        6: 자전거 하차
    '''
    tmp_url = url + '/simulate'
    return requests.put(tmp_url, headers = {'Authorization' : auth_key}, json = {"commands" : commands})

def score(auth_key):
    tmp_url = url + '/score'
    return requests.get(tmp_url, headers = {'Authorization' : auth_key})

def change_loc(N,i,j):
    return j, N-i

def next_loc(N, _id, idx):
    i,j = idx[0],idx[1]
    # 지그재그식
    if i == N-1 and j == N-1:
        return -1
    if j % 2 == 1:
        if i == N-1:
            return 2
        else:
            return 3
    if j % 2 == 0:
        if i == 0:
            return 2
        else:
            return 1
        
def next_loc_idx(num,idx):
    i,j = idx[0],idx[1]
    if num == 1: return [i-1,j]
    elif num == 2: return [i, j+1]
    elif num == 3: return [i+1, j]
    elif num == 4: return [i, j-1]

def main():
    token = {'X-Auth-Token' : '715f565304b4aaab9d6de793bfc51818'}
    N = 60
    truck = {_ : [59,_*5] for _ in range(10)}
    
    start_res = start(token, 2)
    print(start_res)
    
    locations_res = locations(start_res['auth_key'])
    # 각 자전거 대여소의 ID, 보유하고 있는 자전거 수에 대한 정보를 담은 배열
#     print(locations_res.json())
    
    trucks_res = trucks(start_res['auth_key'])
    # 각 트럭의 ID, 현재 위치, 싣고 있는 자전거 수에 대한 정보를 담은 배열
    print(trucks_res.json())
    
    # 처음 1분은 열별 1대씩 배치
    for _ in range(2):
        simulate_res = simulate(start_res['auth_key'], commands = [{"truck_id": 1, "command": [2],
                                                                   "truck_id": 2, "command": [2,2],
                                                                   "truck_id": 3, "command": [2,2,2],
                                                                   "truck_id": 4, "command": [2,2,2,2],
                                                                   "truck_id": 5, "command": [2,2,2,2,2],
                                                                   "truck_id": 6, "command": [2,2,2,2,2,2],
                                                                   "truck_id": 7, "command": [2,2,2,2,2,2,2],
                                                                   "truck_id": 8, "command": [2,2,2,2,2,2,2,2],
                                                                   "truck_id": 9, "command": [2,2,2,2,2,2,2,2,2]}])
        

    # 240 3037 => 3595
    # [22,50],[4,59]
    # 480 2635 => 724
    # [59,8],[4,43]
    # 2465 => 969
    # [54,41], [50,16]
    hot_place = [[22,50],[4,59], [59,8],[4,43], [54,41], [50,16]]
    
    for _ in range(720):
        if _ < 240: idx = 0
        elif _ < 480: idx = 1
        else: idx = 2
            
        _dict = [{'truck_id' : _, "command" : []} for _ in range(10)]
        for _ in _dict:
            _['command'] = []
            next_res = next_loc(N, _['truck_id'], truck[_['truck_id']])
            if next_res == -1:
                _['command'] += [4] * 6
                truck[_['truck_id']] = [0,54]
            else:
                if truck[_['truck_id']] == hot_place[idx][0]:
                    _['command'] += [5] * 8
                    _['command'] += [6] * 1
                elif truck[_['truck_id']] == hot_place[idx][1]:
                    _['command'] += [6] * 10
                
                _['command'] += [next_res]
                truck[_['truck_id']] = next_loc_idx(next_res, truck[_['truck_id']])
                _['command'] += [5] * 2
                _['command'] += [6] * 1
                
                next_res = next_loc(N, _['truck_id'], truck[_['truck_id']])
                _['command'] += [next_res]
                truck[_['truck_id']] = next_loc_idx(next_res, truck[_['truck_id']])
                _['command'] += [5] * 2
                _['command'] += [6] * 1
                
        simulate_res = simulate(start_res['auth_key'],_dict)
        print(simulate_res.json())
    
    score_res = score(start_res['auth_key'])
    print(score_res.json())

In [5]:
if __name__ == '__main__':
    main()

{'auth_key': '0862c189-ffdc-4de8-a70d-3d5e7630d36f', 'problem': 2, 'time': 0}
{'trucks': [{'id': 0, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 1, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 2, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 3, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 4, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 5, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 6, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 7, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 8, 'loaded_bikes_count': 0, 'location_id': 0}, {'id': 9, 'loaded_bikes_count': 0, 'location_id': 0}]}
{'status': 'ready', 'time': 3, 'failed_requests_count': 0, 'distance': '3.8'}
{'status': 'ready', 'time': 4, 'failed_requests_count': 0, 'distance': '5.8'}
{'status': 'ready', 'time': 5, 'failed_requests_count': 0, 'distance': '7.8'}
{'status': 'ready', 'time': 6, 'failed_requests_count': 0, 'distance': '9.8'}


KeyboardInterrupt: 